# Provision Using `nvflare provision` CLI

As mentioned in the [introduction to NVIDIA FLARE provisioning](../04.0_introduction/provision.ipynb), one way of performing provisioning is to use the `nvflare provision` CLI.

Execute the following command to see the documentation for the `nvflare provision` command.

In [ ]:
! nvflare provision -h

Now, let's look at an example of how to provision a project using the `nvflare provision` CLI tool in 3 steps.

# Example: Provision a Federated Project

### 1. Configure the project

The input to the povisioning process is a project configuration file (typically a `yaml` file). This configuration is used to set up the FL project, and generally includes definitions for:
- Project meta-data
- Participants of the project
- Builders to build project workspace

An example project configuration file is provided in [`code/project.yml`](code/project.yml). This file was generated using the `nvflare provision` CLI tool without any arguments. You can also find master templates for more complete project configurations [here](https://nvflare.readthedocs.io/en/main/programming_guide/provisioning_system.html#default-project-yml-file).

> **NOTE**:
> when running this CLI tool without arguments, you will be prompted to select whether the generated configuration file should include [*high-availability*](https://nvflare.readthedocs.io/en/main/programming_guide/high_availability.html) features. We will not cover *high-availability* in this course, and the example file was generated without high-availability features. 

Let us look at the main content of [`code/project.yaml`](code/project.yml) together (some of the details and comments are removed to faciliate explanation):
```yaml
api_version: 3
name: example_project
description: NVIDIA FLARE sample project yaml file

participants:
  - name: server1
    type: server
    org: nvidia
    fed_learn_port: 8002
    admin_port: 8003

  - name: site-1
    type: client
    org: nvidia

  - name: site-2
    type: client
    org: nvidia
    
  - name: admin@nvidia.com
    type: admin
    org: nvidia
    role: project_admin

builders:
  - path: nvflare.lighter.impl.workspace.WorkspaceBuilder
      ...
  - path: nvflare.lighter.impl.static_file.StaticFileBuilder
      ...
  - path: nvflare.lighter.impl.cert.CertBuilder
      ...
  - path: nvflare.lighter.impl.signature.SignatureBuilder
```

The configuration file is organized into the following 3 sections:

**Project meta-data**
  - The `api_version`: for current release of NVIDIA FLARE, the `api_version` is set to 3
  - The `name` of the FL project, in this case, `example_project`
  - And a short `description` of the FL project

**Participants**

This section is a crucial part that defines all the [participants](https://nvflare.readthedocs.io/en/main/programming_guide/provisioning_system.html#participant) involved in the project. There are multiple types of participants defined in FLARE, most common types include `server`, `client` and `admin`. Some participants are referred to as [sites](https://nvflare.readthedocs.io/en/main/user_guide/security/terminologies_and_roles.html#site), which represent computing systems that run FLARE applications, for instance, the `server` and `client`. While some participants are referred to as [users](https://nvflare.readthedocs.io/en/main/user_guide/security/terminologies_and_roles.html#user), who are human participants, with different access priviledges to query, monitor or manage the project, for instance, the `admin`. Developers can also include other types, like the `overseer` for high-availability mode, or add custom types - we will not cover the details of these advanced features in this course.

As we can see, in the `project.yml` file, the following participants are defined:
- 1 `server`: the name of the server should in general be a [fully qualified domain name](https://en.wikipedia.org/wiki/Fully_qualified_domain_name) (FQDN) to make sure that other participants can establish network connections to the server from any location. The server name can also be a system-wide known hostname.
- 2 `client`s with names `site-1` and `site-2`.
- 1 `admin`: here we defined 1 `project_admin` with name `admin@nvidia.com`, who has the most elevated access priviledges in the whole project. There can only 1 project admin for each project.

**Builders**

In NVIDIA FLARE provisioning, [builders](https://nvflare.readthedocs.io/en/main/programming_guide/provisioning_system.html#builder) are a series of Python classes that work together to generate startup kits for all participants, based on the configurations defined in the project configuration file. Builders create various aspects of the startup kits for all participants, such as workspace structure, configuration files, and security credentials. Builders specified in a configuration file will be invoked in the same order as listed in the `builders` section. This example `project.yml` file shows the usage of common builders:
- `WorkspaceBuilder`: Creates the basic workspace structure
- `StaticFileBuilder`: Copies static files into the workspace
- `CertBuilder`: Generates certificates and keys for secure communication
- `SignatureBuilder`: Creates signatures for tamper-proofing

Developers can also create custom builders to construct more specific and customized provisioning output. We will not deep dive into how builders work and how they can be customized. For now, the most important thing to know about builders is that they execute in specific order to build the startup kits as results of a provisioning process.

## 2. Generate startup kits

Let's run the provisioning process with the example configuration file [`code/project.yml`](code/project.yml/):

In [ ]:
!rm -rf ./workspace
!nvflare provision -p code/project.yml -w ./workspace

Now let's check the structure of the output generated by the `nvflare provision` command:

In [ ]:
!tree ./workspace -L 4

We can see that provisioning generates a well-defined directory hierarchy.

At the top level, we have a root folder with the FL project's name, in this case, `example_project`. Under `example_project`, we can see multiple subfolders:
- `resources`: this directory may contain shared resources or additional files needed for the project.
- `state`: this is a directory to maintain state information about the provisioning process or current status of participants.
- `prod_NN`: this folder contains the **startup kits** generated from a successful provisioning command for all participants. The number `NN` increases with each successful provision run, indicating different provisioning sessions. In this example case, since we are running the provisioning command for the first time, the folder name is `prod_00`.

**A startup kit is a folder with a set of files, scripts and credentials generated by provisioning for each participant in the FL project**. A startup kit is local to a specific participant and typically includes:
- Authentication credentials
- Authorization policies
- Signatures for tamper-proof mechanisms
- Convenient shell scripts for launching the participant

We can see that in this example, there are 4 startup kits generated, one for each participants, i.e., 1 server, 2 clients and 1 admin. Each folder is named after its corresponding participant's name as indicated in the configuration file. 

Let's look into the content of the generated startup kits for the server, clients and admin. 

Server and clients have similar startup kit content structure, so let's just display the server startup kit files with the following command:


In [ ]:
!tree ./workspace/example_project/prod_00/server1/

Different subfolders are organized as follows: 
- The `local` subfolder: this folder contains configuration files for site-specific local policies for authorization, logging, privacy and resource access. Each site can modify these configuration files to set up their local policies. We will see more details on local site policy management in the next chapter.
- The `startup` subfolder: this folder contains shell script for a participant to start / join (`start.sh`) or leave the FL project (`stop_fl.sh`). It also contains certificates, signature and key files, which are essential to maintain secure connections between different participants. Note that the signatures and certificates files are integral to ensuring the security and authenticity of a participant. Manual modification of these files after provisioning may prevent a participant to connect to the FL system. 
- The `transfer` subfolder is used to store artifacts such as custom application, scripts or files during project runtime.

Now let's look at the content of `admin`'s startup kit:

In [ ]:
!tree ./workspace/example_project/prod_00/admin@nvidia.com/

The `admin`'s startup kit has similar structure. 
- The `local` subfolder is empty, since local policies are not needed for `admin`, a human participant who manages the project.
- In the `startup` subfolder, apart from certificates and keys, the `fl_admin.sh` shell script allows the `admin` user to login to [**FLARE Console**](https://nvflare.readthedocs.io/en/main/user_guide/admin_guide/deployment/operation.html#operating-nvflare) to perform management tasks from anywhere with secure network connection. We will look at the **FLARE Console** later.
- The `transfer` folder in `admin`'s startup kit is the default location to hold various runtime data, for instance, downloads of federated jobs from the server workspace when they finish. It can also be used to link or hold federated applications developed using FLARE, so that the `admin` user can easily submit the application to the server for running. We will see how this works in details later.

### 3. Distribute the startup kits to participants

Now with startup kits generated, the last step is to distribute these kits to the corresponding participants. You can use email, sftp etc. to do so, as long as you can ensure that it is secure. In general, each site should have an organization admin to receive or download the startup kits. The organization admin can then install their own packages, start the services, map the data location, and instrument the authorization policies and organization level site privacy policies. Another way to distribute startup kits is to use the [FLARE Dashboard](https://nvflare.readthedocs.io/en/main/user_guide/dashboard_ui.html) with a convenient webUI. We will briefly cover the FLARE Dashboard in the next chapter.


**That's it! We have learned how to provision a federated project and generate startup kits for all participants using the `nvflare provision` CLI tool!**

# What's Next

Next, let's learn the other option for provisioning: [FLARE Dashboard](../04.2_provision_via_dashboard/provision_via_dashboard.ipynb).